# WebRender dashboard ETL script

You probably want to look at the [published dashboard](file:///Users/tsmith/projects/webrender-dashboard/dashboard.html).

You can check [scheduled run status](https://dbc-caf9527b-e073.cloud.databricks.com/#job/715).


### What's different?

* Metrics are summarized by build (vs date).
* Performance metrics are aggregated over users; this is important because it reduces the impacts of outlier users on our understanding of product performance, and because it reflects how WebRender changes the user experience at the user level.
* Pings are artisanally hand-selected so that comparisons between experiment branches are fair even in the presence of the various enrollment weirdnesses.

In [2]:
%r
library(boot)
library(dplyr, warn.conflicts=FALSE)
library(ggplot2)
library(sparklyr, warn.conflicts=FALSE)
library(tidyr, warn.conflicts=FALSE)

sc = spark_connect(method="databricks")

# Save result tables so we can access them from hala


In [3]:
from moztelemetry.dataset import Dataset
import pandas as pd
from pyspark.sql import Row
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, DoubleType, LongType, MapType
from statsmodels.stats.weightstats import DescrStatsW

In [4]:
  Dataset.from_source("telemetry").schema

Out[ 19 ]: 
[u'submissionDate',
 u'sourceName',
 u'sourceVersion',
 u'docType',
 u'appName',
 u'appUpdateChannel',
 u'appVersion',
 u'appBuildId']

In [5]:


pings = (
  Dataset
  .from_source("telemetry")
  .where(
    docType="main",
  )
  .where(appUpdateChannel="beta")
  .where(submissionDate='20181220')
  .select(
    app_build_id="application.buildId",
    environment='environment.system',
    dwrite_init_problem="payload.histograms.DWRITEFONT_INIT_PROBLEM",
    client_id="clientId",
    normalized_channel="meta.normalizedChannel",
    profile_subsession_counter="payload.info.profileSubsessionCounter",
    session_id="payload.info.sessionId",
  )
  .records(sc, sample=0.1)
)

WARNING: THIS IS NOT A REPRESENTATIVE SAMPLE.
This 'sampling' is based on s3 files and is highly
susceptible to skew. Use only for quicker performance
while prototyping.
fetching 17368.29883MB in 9745 files...

In [6]:
#def 
pings = pings.filter(lambda p: p["dwrite_init_problem"])
pings.cache()
pings.take(1)

Out[ 33 ]: 
[{'app_build_id': u'20181217180946',
 'client_id': u'cf2e8164-6b20-442f-87d1-4d5ed20ccfc4',
 'dwrite_init_problem': {u'bucket_count': 9,
 u'histogram_type': 1,
 u'range': [1, 8],
 u'sum': 2,
 u'values': {u'1': 0, u'2': 1, u'3': 0}},
 'environment': {u'appleModelId': None,
 u'cpu': {u'cores': 2,
 u'count': 4,
 u'extensions': [u'hasMMX',
 u'hasSSE',
 u'hasSSE2',
 u'hasSSE3',
 u'hasSSSE3',
 u'hasSSE4_1',
 u'hasSSE4_2',
 u'hasAES'],
 u'family': 6,
 u'l2cacheKB': 256,
 u'l3cacheKB': 3072,
 u'model': 60,
 u'speedMHz': 3392,
 u'stepping': 3,
 u'vendor': u'GenuineIntel'},
 u'gfx': {u'ContentBackend': u'Skia',
 u'D2DEnabled': False,
 u'DWriteEnabled': True,
 u'adapters': [{u'GPUActive': True,
 u'RAM': None,
 u'description': u'Intel(R) HD Graphics 4400',
 u'deviceID': u'0x041e',
 u'driver': u'igdumdim32 igd10iumd32 igd10iumd32',
 u'driverDate': u'3-30-2014',
 u'driverVersion': u'10.18.10.3540',
 u'subsysID': u'00000000',
 u'vendorID': u'0x8086'}],
 u'features': {u'advancedLayers': {u'status': u'available'},
 u'compositor': u'd3d11',
 u'd2d': {u'status': u'failed', u'version': u'1.1'},
 u'd3d11': {u'blacklisted': False,
 u'status': u'available',
 u'textureSharing': True,
 u'version': 45056,
 u'warp': False},
 u'gpuProcess': {u'status': u'unavailable'},
 u'webrender': {u'status': u'unavailable'},
 u'wrQualified': {u'status': u'blocked'}},
 u'monitors': [{u'pseudoDisplay': False,
 u'refreshRate': 60,
 u'screenHeight': 768,
 u'screenWidth': 1366}]},
 u'hdd': {u'binary': {u'model': u'ATA Hitachi HCS5C101',
 u'revision': u'A620'},
 u'profile': {u'model': u'ATA Hitachi HCS5C101', u'revision': u'A620'},
 u'system': {u'model': u'ATA Hitachi HCS5C101', u'revision': u'A620'}},
 u'isWow64': False,
 u'memoryMB': 3494,
 u'os': {u'installYear': 2016,
 u'locale': u'en-US',
 u'name': u'Windows_NT',
 u'servicePackMajor': 0,
 u'servicePackMinor': 0,
 u'version': u'6.1',
 u'windowsBuildNumber': 7600},
 u'virtualMaxMB': 2048},
 'normalized_channel': u'beta',
 'profile_subsession_counter': 351,
 'session_id': u'c7986b8b-2640-4471-8e60-57c25f359302'}]

In [7]:
pings.filter(lambda p: 'windowsBuildNumber' in p['environment']['os']).map(lambda p: p['environment']['os']['windowsBuildNumber']).countByValue()

Out[ 40 ]: defaultdict(<type 'int'>, {7600: 33, 7601: 29})

In [8]:
pings.count()

Out[ 41 ]: 63